### Data Access

In [0]:
storage_account = "nyctaxistoragejubin"
access_key = "sample"
 
spark.conf.set(
  f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
  access_key
)

### Database Creation

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold

### Data Reading, writing and creating delta tables

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
silver = 'abfss://silver@nyctaxistoragejubin.dfs.core.windows.net'
gold = 'abfss://gold@nyctaxistoragejubin.dfs.core.windows.net'

#### Data Zone

In [0]:
df_zone = spark.read.format('parquet')\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(f"{silver}/trip_zone")  

In [0]:
df_zone.display()

In [0]:
df_zone.write.format('delta')\
    .mode('append')\
    .option('path', f'{gold}/trip_zone')\
    .save()
df_zone.write.format("delta").saveAsTable("default.trip_zone")

#### Data type

In [0]:
df_type = spark.read.format('parquet')\
               .option('header', True)\
               .option('inferSchema', True)\
               .load(f"{silver}/trip_type")

In [0]:
df_type.display()

In [0]:
df_type.write.format('delta')\
                    .mode('append')\
                    .option('path', f'{gold}/trip_type')\
                    .save()
df_type.write.format("delta").saveAsTable("default.trip_type")

#### Trips date

In [0]:
df_trip = spark.read.format('parquet')\
               .option('header', True)\
               .option('inferSchema', True)\
               .load(f"{silver}/trip2023data")

In [0]:
df_trip.display()

In [0]:
df_trip.write.format('delta')\
                    .mode('append')\
                    .option('path', f'{gold}/tripsdata')\
                    .save()
df_trip.write.format("delta").saveAsTable("default.tripsdata")

In [0]:
df_trip.display()

### Learning Delta Lake

#### Versioning

In [0]:
%sql
select * from default.trip_zone where LocationID = 1

In [0]:
%sql
UPDATE default.trip_zone 
SET Borough = 'EMR' where LocationID = 1;

In [0]:
%sql
delete from default.trip_zone where LocationID = 1

In [0]:
%sql
describe history default.trip_zone

In [0]:
%sql
select * from default.trip_zone where LocationID = 1

#### Roll back

In [0]:
%sql
restore default.trip_zone to version as of 0


In [0]:
%sql
select * from default.trip_zone where LocationID = 1

### Delta tables

#### Trip type

In [0]:
%sql
select * from default.trip_type

#### Trip Zone

In [0]:
%sql
select * from default.trip_zone

#### Trip Data

In [0]:
%sql
select * from default.tripsdata